In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Define a Stratey:
# Short/Long Exponential Moving Average Crossover strategy with
# 'crossover factor' that adjust the buy/sell crossover threshold

class TestStrategy(bt.Strategy):
    params = (
        ('expSperiod', 15),
        ('expLperiod', 30),
        ('cross_fac', 1.0),
        ('printlog', False),
        ('printstop', False)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open

        # To keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a MovingAverageSimple indicator
        self.ema_short = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.expSperiod
                         )
        self.ema_long = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.expLperiod
                         )
        
        self.xf = self.params.cross_fac

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm)
                )
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            
            elif order.issell():
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm)
                )

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Open: %.2f, Close: %.2f' % (self.dataopen[0], self.dataclose[0]))

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.ema_short[0]*self.xf < self.ema_long[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.ema_short[0]*self.xf >= self.ema_long[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        if self.params.printstop:
            self.log('(S Period %2d, L Period %2d, XF %.2f) Ending Value %.2f' %
                     (self.params.expSperiod, self.params.expLperiod, 
                      self.params.cross_fac, self.broker.getvalue()), doprint=True)
            

In [ ]:
class EMAAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.expSper = self.strategy.params.expSperiod
        self.expLper = self.strategy.params.expLperiod
        self.xf = self.strategy.params.cross_fac
        
    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        self.ntrade += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        outp = {
            'params': (self.expSper, self.expLper, self.xf),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade
        }
        
        return outp
    

In [ ]:
date_i_train = datetime.datetime(1995, 1, 3)
date_f_train = datetime.datetime(2010, 12, 31)
date_i_test  = datetime.datetime(2010, 1, 1)
date_f_test  = datetime.datetime(2014, 12, 31)

## Optimize

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.optstrategy(
            TestStrategy,
            expSperiod=range(3,8),
            expLperiod=range(30,91,3),
            cross_fac =np.linspace(0.9,1.1,11))
cerebro.optreturn = False
#cerebro.addstrategy(TestStrategy)

# Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(EMAAnalyzer, _name='ema')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=date_i_train,
            # Do not pass values before this date
            todate=date_f_train,
            reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.00)

#slippage
cerebro.broker.set_slippage_fixed(.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)

# Run over everything
result = cerebro.run(maxcpus=1)

Collect optimization results and plot:

In [ ]:
params  = []
n_trades = []
profit   = []
for res in result:
    r = res[0].analyzers.ema.get_analysis()
    params.append(r['params'])
    n_trades.append(r['ntrade'])
    profit.append(r['profit'])

In [ ]:
fig, (prof, nt) = plt.subplots(1,2, figsize=(15,5))
_=prof.hist(profit, bins=np.linspace(-150,450,50))
_=nt.hist(n_trades, bins=np.arange(0,200,2))

prof.set_title('Profit Distribution')
nt.set_title('# of Trades Distribution')



Find 'best' strategy by most profit

In [ ]:
prof_ind = np.argmax(profit) 
best_params = params[prof_ind]
print(best_params, profit[prof_ind], n_trades[prof_ind])

## Run with 'best' setting in-sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, 
                    expSperiod=4, 
                    expLperiod=45,
                    cross_fac = 1.00,
                    printlog=False,
                    printstop=True)

# Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(EMAAnalyzer, _name='ema')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=date_i_train,
            # Do not pass values before this date
            todate=date_f_train,
            reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.00)

#slippage
cerebro.broker.set_slippage_fixed(.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)

# Run over everything
result = cerebro.run()
in_samp_sharpe = result[0].analyzers.mysharpe.get_analysis()
in_samp_drawdown = result[0].analyzers.drawdown.get_analysis()

In [ ]:
#%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
cerebro.plot(plotabove=True)

## Run with 'best' settings out-of-sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, 
                    expSperiod=4, 
                    expLperiod=45,
                    cross_fac = 1.00,
                    printlog=False,
                    printstop=True)

# Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(EMAAnalyzer, _name='ema')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=date_i_test,
            # Do not pass values before this date
            todate=date_f_test,
            reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.00)

#slippage
cerebro.broker.set_slippage_fixed(.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)

# Run over everything
result = cerebro.run(maxcpus=1)
out_samp_sharpe = result[0].analyzers.mysharpe.get_analysis()
out_samp_drawdown = result[0].analyzers.drawdown.get_analysis()

In [ ]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
cerebro.plot(plotabove=True)

### Compare in vs. out

In [ ]:
print('in sample:')
print(in_samp_sharpe)
#print(in_samp_drawdown)
print('\n out sample:')
print(out_samp_sharpe)
#print(out_samp_drawdown)